#### Clinvarに関するコード
ファイルの名称について
- NPをACに変換
    - uniprot_NP2AC.json
- ClinvarのNP（NCBIのReference sequenceにおけるタンパク質のアセッション番号）をUniprotのACに変換し、jsonにしたもの
    - clinvar_noheader.json
- 最初のACエントリだけ採用し、病名表記をDiseaseとPolymorphismに規格化したもの
    - clinvar_1ac.json


In [ ]:
import re
import json
from collections import defaultdict

In [ ]:
clinvar_data = open("../clinvar_231230_noheader.txt")

In [ ]:
#RefSeqのNPをACに変換する
UniprotNP = defaultdict(list)
acs =[]
NP_data = []

pattern = "NP_\d*\.\d"
repatter = re.compile(pattern)

data = open("../uniprot_human.dat")
for line in data:
    if line.startswith("AC"):
        info = line.rstrip().split()
        for b in info[1:]:
            acs.append(b.replace(";",""))  
    if line.startswith("DR   RefSeq; NP"):
        info2 = line.split()
        NP_data.append(info2[2].replace(";",""))
    if line.startswith("DR   MANE-Select;"):
        np = repatter.findall(line)
        if np is not None:
            NP_data.append(np[0])
    elif line.startswith("//"):
        for a in NP_data:
            UniprotNP[a]=acs
        acs = []
        NP_data = []

with open('../uniprot_NP2AC.json', 'w') as f:
    json.dump(UniprotNP, f, indent=4)

In [ ]:
#acを全部入れる
Uniprot_NP2AC = json.load(open("../uniprot_NP2AC.json"))

Uniprot_NP2AC_ids = []
for data in Uniprot_NP2AC:
    Uniprot_NP2AC_ids.append(data)
Clinvar_dict = defaultdict(list)
notinUniprot = []
for line in clinvar_data:
    entries = line.split("	")
    if entries[2] in Uniprot_NP2AC_ids:
        acs = Uniprot_NP2AC[entries[2]]
        data = {"acs":acs,"Variant":{"before":entries[3][:3],"after":entries[3][-3:]},"Position":entries[4],"AlphaMissense":entries[5],"Grantham":entries[6],"Sequence_flanking":entries[7],"pLDDT_mean":entries[8],"Clinical_significance":entries[9:]}
        Clinvar_dict[acs[0]].append(data)
    else:
        notinUniprot.append(entries[2])
print(notinUniprot)

with open('../clinvar_noheader.json', 'w') as f:
    json.dump(Clinvar_dict, f, indent=4)

分類方針
1. benignが含まれる場合、Polymorphism
2. pathogenicが含まれる場合、Disease

In [ ]:
#acは最初のエントリだけ
Clinvar_dict2 = defaultdict(list)

for line in clinvar_data:
    entries = line.split("	")
    if entries[2] in Uniprot_NP2AC_ids:
        if "pathogenic" in entries[9:][0].lower():
            acs = Uniprot_NP2AC[entries[2]]
            data = {"Variant":{"before":entries[3][:3],"after":entries[3][-3:]},"Position":entries[4],"AlphaMissense":entries[5],"Grantham":entries[6],"Sequence_flanking":entries[7],"pLDDT_mean":entries[8],"Clinical_significance":"Disease"}
            Clinvar_dict2[acs[0]].append(data)
        if "benign" in entries[9:][0].lower():
            acs = Uniprot_NP2AC[entries[2]]
            data = {"Variant":{"before":entries[3][:3],"after":entries[3][-3:]},"Position":entries[4],"AlphaMissense":entries[5],"Grantham":entries[6],"Sequence_flanking":entries[7],"pLDDT_mean":entries[8],"Clinical_significance":"Polymorphism"}
            Clinvar_dict2[acs[0]].append(data)
        

with open('../clinvar_1ac.json', 'w') as f:
    json.dump(Clinvar_dict2, f, indent=4)